# The Enron Email Dataset

## 1. Downloading data

More info about dataset: [link](https://www.kaggle.com/datasets/wcukierski/enron-email-dataset)

Please provide your Kaggle credentials to download this dataset. Learn more: [link](http://bit.ly/kaggle-creds)

In [1]:
import pandas as pd
import numpy as np

import opendatasets as od

od.download_kaggle_dataset("https://www.kaggle.com/datasets/wcukierski/enron-email-dataset", data_dir='.')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading enron-email-dataset.zip to ./enron-email-dataset


100%|██████████| 358M/358M [07:55<00:00, 789kB/s]  


In [2]:
data = pd.read_csv('enron-email-dataset/emails.csv')
data.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


## 2. Parse data